![Banner](../../assets/banner-sprint7-desafio.png)



## IMPORTAÇÕES E LEITURA DO DATASET

Importação de bibliotecas para análise e tratamento dos dados.

In [1]:
import pandas as pd
import re
from transformers import pipeline

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Leitura do dataset para um DataFrame, com configurações específicas para o formato.



In [2]:
df = pd.read_csv('../../sprint6/desafio/dados_raw/movies.csv',
    delimiter='|',
    quotechar='"',
    escapechar='\\',
    engine='python'
)

Visualização de amostra.

In [3]:
df.head()

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
0,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actress,Miss Geraldine Holbrook (Miss Jerry),Blanche Bayliss,1878,1951,actress,tt0000009
1,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Mr. Hamilton,William Courtenay,1875,1933,actor,"tt0021535,tt0000009,tt0020355,tt0020403"
2,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Chauncey Depew - the Director of the New York ...,Chauncey Depew,1834,1928,"actor,writer","tt1076833,tt0490842,tt0000009,tt4484306"
3,tt0000502,Bohemios,Bohemios,1905,100,N,4.2,14,actor,N,Antonio del Pozo,N,N,actor,tt0000502
4,tt0000502,Bohemios,Bohemios,1905,100,N,4.2,14,actor,N,El Mochuelo,N,N,actor,tt0000502


### Filtro 1: Gênero

Dentre os gêneros designados, foi decidido manter somente `Romance` para um recorte mais específico.

In [3]:
df1 = df[df['genero'].str.contains(r'^Romance$', regex=True)]
df1.shape

(11919, 15)

### Filtro 2: Ano 2013 ~ Atual

Recorte referente ao período de popularização da Netflix, abarcando o início de suas atividades fora dos EUA, a expansão e presença global, até os dias atuais.

Primeiramente, foi realizada a tratativa de possíveis valores nulos na coluna `anoLancamento` e, depois, conversão da coluna para `int` .

In [4]:
df1.loc[df1['anoLancamento'].str.contains(r'^[a-zA-Z]+$', regex=True), 'anoLancamento'] = 0
df1['anoLancamento'] = df1['anoLancamento'].astype(int)

/tmp/ipykernel_4295/1949478750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['anoLancamento'] = df1['anoLancamento'].astype(int)


Confirmação da conversão.

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11919 entries, 0 to 1044551
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     11919 non-null  object 
 1   tituloPincipal         11919 non-null  object 
 2   tituloOriginal         11919 non-null  object 
 3   anoLancamento          11919 non-null  int64  
 4   tempoMinutos           11919 non-null  object 
 5   genero                 11919 non-null  object 
 6   notaMedia              11919 non-null  float64
 7   numeroVotos            11919 non-null  int64  
 8   generoArtista          11919 non-null  object 
 9   personagem             11919 non-null  object 
 10  nomeArtista            11919 non-null  object 
 11  anoNascimento          11919 non-null  object 
 12  anoFalecimento         11919 non-null  object 
 13  profissao              11830 non-null  object 
 14  titulosMaisConhecidos  11918 non-null  object 
dtypes: fl

Recorte por período, este DataFrame será a base para inserção dos dados obtidos via API, visto que serão utilizadas as linhas com atrizes/atores distintos para a análise.

In [5]:
df2 = df1[df1['anoLancamento'] >= 2013]
df2.shape

(3975, 15)

Download deste DataFrame para utilização futura.

In [8]:
df2.to_csv(
    './csv/dataset_base_com_elenco.csv',
    sep=',',
    header=True,
    index=False
)

### Filtro 3: Filtrar Filmes Distintos

Este é o filtro utilizado para obter somente os IDs distintos dos filmes interesse para as requisições via API.

In [6]:
df3 = df2.drop_duplicates(subset=['id'], keep='last')
df3.shape

(920, 15)

#### Inserção de Coluna de Título Original em Inglês (Boolean)

Tentativa de filtrar excluindo as linhas com títulos originais em inglês, a partir de utilização de modelo de linguagem para reconhecimento de língua.

Apesar da classificação correta, alguns títulos da coluna `tituloOriginal` não estão na língua correta, e sim em inglês.

Portanto, este filtro foi inutilizado.

In [12]:
detector_lingua = pipeline("text-classification",
                           model="papluca/xlm-roberta-base-language-detection")

def ingles(texto):
    classificacao = detector_lingua(texto)
    return classificacao[0]['label'] == 'en'

# df3['emIngles'] = df3['tituloOriginal'].apply(ingles)
df3.loc[:, ['id', 'tituloOriginal', 'emIngles']]

Device set to use cpu


,id,tituloOriginal,emIngles
563534,tt0850678,Jackson Arms,True
576817,tt10006510,Life Mein Twist Hai,False
577446,tt10033852,Rangeela Raybaa,False
577449,tt10033904,Spandan What is Relationship,True
577569,tt10039472,Premak Basaat,False
...,...,...,...
1042395,tt9743420,Tinh dau tho ngay,False
1042691,tt9765480,Elise,False
1042761,tt9766610,Cho Em Gan Anh Them Chut Nua,False
1043920,tt9840958,Love Struck Sick,True


Download do arquivo gerado, com filmes distintos e coluna de classificação de `tituloOriginal` .

In [13]:
df3.to_csv(
    './csv/ids_distintos_attr_em_ingles.csv',
    sep=',',
    header=True,
    index=False
)